LINQ provides `two parallel architectures`: local queries for local object collections and interpreted queries for remote data sources.

***Local queries*** resolve to `query operators` in the `Enumerable` class

interpreted queries are `descriptive`(توصیفی). They operate over sequences that
implement `IQueryable<T>`, and they resolve to the query operators in the `Queryable` class

***interpreted queries*** use `expression trees` to translated `IQueryable` queries to for example `SQL queries`.

To `write` interpreted queries, you need to start with an `API` that e`xposes sequences of type IQueryable<T>`. Like `EFCore`.

It’s also possible to `generate an IQueryable<T>` wrapper around an ordinary `enumerable collection` by calling the `AsQueryable` method.

In [4]:
#r "nuget:Microsoft.EntityFrameworkCore"

Installed Packages Microsoft.EntityFrameworkCore, 8.0.4

In [5]:
using System.Linq;
using Microsoft.EntityFrameworkCore;

public class Customer
{
    public int ID { get; set; }
    public string Name { get; set; }
}

public class NutshellContext : DbContext
{
    public virtual DbSet<Customer> Customers { get; set; }
    protected override void OnConfiguring (DbContextOptionsBuilder builder)
        => builder.UseSqlServer ("...connection string...");
    protected override void OnModelCreating (ModelBuilder modelBuilder)
        => modelBuilder
            .Entity<Customer>()
            .ToTable ("Customer")
            .HasKey (c => c.ID);
}

using var dbContext = new NutshellContext();
IQueryable<string> query = from c in dbContext.Customers
where c.Name.Contains ("a")
orderby c.Name.Length
select c.Name.ToUpper();

Error: (22,11): error CS1002: ; expected

In [ ]:
--EF Core translates this query into the following SQL:
SELECT UPPER([c].[Name])
FROM [Customers] AS [c]
WHERE CHARINDEX(N'a', [c].[Name]) > 0
ORDER BY CAST(LEN([c].[Name]) AS int)

### How Interpreted Queries Work

> ***First***, the compiler converts `query syntax` to `fluent syntax`.  
  
> ***Next***, the compiler resolves interpreted queries to query operators in the `Queryable` class `instead of` the `Enumerable` class

In [ ]:
public static IQueryable<TSource> Where<TSource> (this
IQueryable<TSource> source, Expression <Func<TSource,bool>> predicate)

> when `compiler` see ***Where***, choice use `Where` extension method in `Queryable`, beacuse `dbContext.Customers` is of type `DbSet<T>`, which implements `IQueryable<T>` (a subtype of IEnumerable<T>).

> An `expression tree` is an object model based on the types in `System.Linq.Expressions` that can be `inspected` at `runtime` (so that `EF Core` can later `translate` it to an SQL statement).

> `Queryable.Where` accepts a predicate wrapped in an `Expression<TDelegate>` type.

### Execution

`Interpreted queries` follow a `deferred(به تعویق افتاده) execution` model. This means that the `SQL statement` is not generated `until` you `start enumerating` the query.

In [ ]:
//enumerating the same query twice results in the database being queried twice.

IQueryable<string> query = from c in dbContext.Customers
where c.Name.Contains ("a")
orderby c.Name.Length
select c.Name.ToUpper();

foreach (var item in query) // execute 1
    Console.WriteLine(item);

foreach (var item in query) //execute 2
    Console.WriteLine(item);

### Combining Interpreted and Local Queries

A query can include both `interpreted` and `local` operators.

A typical pattern is to have the ***local operators*** on the `outside` and the ***interpreted components*** on the `inside`;

In [ ]:
public static IEnumerable<string> Pair (this IEnumerable<string> source)
{
    string firstHalf = null;
    foreach (string element in source)
        if (firstHalf == null)
            firstHalf = element;
        else
        {
            yield return firstHalf + ", " + element;
            firstHalf = null;
        }
}

using(var dbContext = new NutshellContext ())
{
    IEnumerable<string> q = dbContext.Customers
        .Select (c => c.Name.ToUpper())
        .OrderBy (n => n)
        .Pair() // Local from this point on.
        .Select ((n, i) => "Pair " + i.ToString() + " = " + n);

    foreach (string element in q) Console.WriteLine (element);
}

In [ ]:
-- this query select of database

SELECT UPPER([c].[Name]) 
FROM [Customers] AS [c] 
ORDER BY UPPER([c].[Name])

--The remaining work is done locally.

### AsEnumerable

`Enumerable.AsEnumerable` is the ***simplest*** of all query operators.

In [ ]:
public static IEnumerable<TSource> AsEnumerable<TSource>
(this IEnumerable<TSource> source)
{
    return source;
}

Its purpose is to ***cast*** an `IQueryable<T>` sequence to `IEnumerable<T>`  
This causes the `remainder` of the query to `execute locally`.

In [ ]:
Regex wordCounter = new Regex (@"\b(\w|[-'])+\b");

using (var dbContext = new NutshellContext ())
{
    var query = dbContext.MedicalArticles
        .Where (article => article.Topic == "influenza" &&
                        wordCounter.Matches (article.Abstract).Count < 100);

    //SQL Server doesn’t support regular expressions, so EF Core
    //will throw an exception
}

In [ ]:
Regex wordCounter = new Regex (@"\b(\w|[-'])+\b");

using (var dbContext = new NutshellContext ())
{
    var query = dbContext.MedicalArticles
        .Where (article => article.Topic == "influenza")
        .AsEnumerable() // reminder query execute locally
        .Where (article => wordCounter.Matches (article.Abstract).Count < 100);
}

An alternative to calling `AsEnumerable` is to call `ToArray` or `ToList`. The `advantage of AsEnumerable` is  
- that it doesn’t force immediate query execution  

-  does it create any storage structure.